##Explore: himalaya.bronze.deaths
Goal: Understand the structure, quality, and issues in the deaths dataset before Silver transformation.

###_Previewing Data_

In [0]:
import pyspark.sql.functions as F

In [0]:
df = spark.table("himalaya.bronze.deaths")

In [0]:
df.show(10)

In [0]:
print(f"Rows: {df.count()}")
df.printSchema()

###_Finding Errors_
1. Null
2. Duplicates
3. Categorical
4. Format
5. Range
6. Abnormals
7. Dates

> ## 1. Nulls

In [0]:
for c in df.columns:
  null_count = df.filter(F.col(c).isNull()).count()
  status = "✅" if null_count == 0 else "❌"
  print(f"{status} Nulls in {c}: {null_count}")

In [0]:
for c in df.columns:
  null_count = df.filter(F.col(c).isNull()).count()
  if null_count == 0:
      print(f"{c}:")
      df.filter(F.col("nationality").isNull()).show()

####Nationality
Can probably find out null values from other data sets.
If not in other data sets, will remove
___
####Cause of Death
No need to find out, it is ok if we don't know how they died

> ## 2. Duplicates

In [0]:
total = df.count()
distinct = df.distinct().count()
duplicates = total - distinct

print(f"Total rows:    {total}")
print(f"Distinct rows: {distinct}")
print(f"Duplicates:    {duplicates}")

if duplicates == 0:
    print("✅ No duplicates")
else:
    print(f"❌ {duplicates} duplicate found")

####